# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
%matplotlib inline

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [ ]:
df=pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')



In [ ]:
df['Year'],df['Month-Day']=zip(*df['Date'].apply(lambda x: (x[:4], x[5:])))
new_df=df[df['Month-Day']!='02-29']
new_df.head()


 

In [ ]:
d_min= new_df[(new_df['Element'] == 'TMIN') & (new_df['Year'] !='2015')]
d_max=new_df[(new_df['Element'] == 'TMAX') & (new_df['Year'] !='2015')]

In [ ]:
d_min_2015=new_df[(new_df['Element'] == 'TMIN') & (new_df['Year'] =='2015')]

d_max_2015=new_df[(new_df['Element'] == 'TMAX') & (new_df['Year'] =='2015')]


In [ ]:
#d_min['Data_Value'].values#head()
min_10_years = d_min.min()
max_10_years = d_max.max()
max_10_years

In [ ]:
mask = (d_max_2015['Data_Value'] > max_10_years['Data_Value'])
data_max = d_max_2015[mask]
data_max['Date']=pd.to_datetime(data_max['Date'],infer_datetime_format=True)
data_max.set_index('Date',inplace=True)



In [ ]:
mask = (d_min_2015['Data_Value'] < min_10_years['Data_Value'])
data_min = d_min_2015[mask]
data_min['Date']=pd.to_datetime(data_min['Date'],infer_datetime_format=True)
data_min.set_index('Date',inplace=True)



In [ ]:
d_min_graph=new_df[new_df['Element'] == 'TMIN']
d_min_graph.reset_index( inplace=False)
d_min_graph.head()

In [ ]:
d_max_graph=new_df[new_df['Element'] == 'TMAX']
d_max_graph.reset_index( inplace=False)
d_max_graph.head()

In [ ]:
ak=d_max_graph.groupby('Date').Data_Value.max()
ak_2=pd.DataFrame(ak)
ak_2.reset_index(inplace=True)
ak_2.rename(columns={'Date':'Date_1','Data_Value':'Data_Value_1'},inplace=True)
ak_2.head()

In [ ]:
ak_3=d_min_graph.groupby('Date').Data_Value.min()
ak_4=pd.DataFrame(ak_3)
ak_4.reset_index(inplace=True)
ak_4.rename(columns={'Date':'Date_1','Data_Value':'Data_Value_2'},inplace=True)
ak_4.head()

In [ ]:
df_cd = pd.merge(ak_2,ak_4, how='outer', left_on = 'Date_1', right_on = 'Date_1')
df_cd.head()



In [ ]:
import numpy as np
date_index = np.arange('2005-01-01','2016-01-01', dtype='datetime64[D]')
date_index

In [ ]:
df_cd['Date_1']=pd.to_datetime(df_cd['Date_1'],infer_datetime_format=True)
df_cd.set_index('Date_1', inplace=True)
df_cd.head()                  


In [ ]:
import numpy as np
plt.figure(figsize=(24,14))
plt.plot(df_cd.index, df_cd.Data_Value_1, color='lightblue',linewidth=1)
plt.plot(df_cd.index,df_cd.Data_Value_2,color='pink', linewidth=1)


plt.scatter(data_max.index,data_max.Data_Value, s = 100, c = 'black')
plt.scatter(data_min.index,data_min.Data_Value, s =100 , c = 'black')


plt.gca().fill_between(((df_cd.index)), 
                       df_cd['Data_Value_2'], df_cd['Data_Value_1'],
                       facecolor='black', 
                       alpha=0.25)
plt.xlabel('Year')
plt.ylabel('Temperature')
plt.title('High and low temperatures by day of the year over the period 2005-2015')

plt.legend(['Maximum Temperature', 'Maximum Temperature','broken low','broken high'])

plt.show()